# Data Proposal
* Ashley Lopez
* Sandra Castro-Valadez
* Alondra Castro-Valadez
* Josue Ramirez

# Project Goal
Our project aims to identify the relationship between different player's stats and their value for when buying a player. We have chosen to focus on the data of 60 players for the last 10 years. Specifically, we ask:

- Can we determine what specific stats clubs should be looking at when buying a player?
- Are there any outside factors that might deter the success of a player after a transfer that clubs should start considering?

We are using https://www.espn.com/soccer/player/stats for our main data source.

# Pipeline Overview
For scraping the soccer teams website:
- get_team_id(team_name)
  - builds a dataframe.....

The are so far the functions we have for our project. It will be utilized with a dictionary of.....

(.5%) Expresses the central motivation of the project and explains the (at least) two key questions to be explored.

(1%) Gives a summary of the data processing pipeline so a technical expert can easily follow along.

(4%) Obtains, cleans, and merges all data sources involved in the project.

(2.5%) Builds at least two visualizations (graphs/plots) from the data which help to understand or answer the questions of interest. These visualizations will be graded based on how much information they can effectively communicate
to readers. Please make sure your visualization are sufficiently distinct from each other.

(2%) Discuss what machine learning tools will be used and the relevant assumptions required to apply each.

In [ ]:
#adding all of the id's for the teams we are using!
team_ids = {'Lech Poznan' : 347, 'Borussia Dortmund' : 165,
            'Fluminense' : 124, 'Real Madrid' : 541, 'Barcelona': 529,
            'Cruz Azul' : 2295,'Feyenoord': 209, 'Manchester United': 33,
            'Monaco' : 91, 'PSG' : 85, 'Tottenham' : 47,
            'Arsenal' : 42, 'Santos': 128, 'Atletico Madrid': 530,
            'Chelsea' : 49, 'Liverpool' : 40, 'Inter Milan':  505,
            'Boca Juniors': 451 , 'Shanghai Shenhua': 833, 'Borussia Dortmund': 165,
            'Athletic Club': 531 , 'Leicester': 46, 'Juventus': 496,
            'AC Milan': 489 , 'Valencia': 532, 'Lazio': 487,
            'Newcastle': 34 , 'Eintracht Frankfurt': 169,
           'Ajax': 425, 'West Ham': 48, 'Sporting CP': 228, 'Eintracht Frankfurt': 169,
           'Roma': 497, 'Bayern Munich': 157, 'Manchester City': 50, 'Everton': 45, 'South Hampton': 41}

In [ ]:
#function that gives you the id of the team based on the name
def get_team_id(team_name):
    id = team_ids.get(team_name)
    return id

cruzAzul = get_team_id('Cruz Azul')

#url = "https://api-football-v1.p.rapidapi.com/v2/fixtures/team/605"


In [ ]:
import requests
import pandas as pd
import json

def get_team_season_players(team_id, season):
    """ Extract json with all players info of a given team and season
        Args: team_id -- int, season (YYYY-YYYY)
    """
    url = f"https://api-football-v1.p.rapidapi.com/v2/players/squad/{team_id}/{season}"
    headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [ ]:
def get_players_dict(team_id, season):
    """ Get dict of all players for given team and season where
         (key, value) -> (player_name, (player_id, p_firstname, p_lastname))
         Args: team_id -- int, season (YYYY-YYYY)
    """
    players_dict = {}
    json = get_team_season_players(team_id, season)['api']['players']
    for i in range(len(json)):
        players_dict[json[i]['player_name']] = (json[i]['player_id'], json[i]['firstname'], json[i]['lastname'])
    return players_dict

In [ ]:
#gets the id of a player based on the name and team, and season u want
def get_player_id(player_name, player_team, year):
    team_id = get_team_id(player_team)
    #from the id let's get the list of all the players
    players_dict = get_players_dict(team_id, year)
    #random id that will be returned if not found
    id = 00000


    for key in players_dict:
        if player_name in key:
            print('FOUND')
            #the value
            value = players_dict[key]
            id = value[0]

    return id


In [ ]:
#testing to see if get_player_id works
get_player_id('Cristiano Ronaldo', 'Manchester United', '2008-2009')

FOUND


874

In [ ]:
#testing to see if it works
get_team_id('Arsenal')

42

In [ ]:
# getting the specific player from a specific id
def id_player_info(id):

  get_id = id

  url = f"https://api-football-v1.p.rapidapi.com/v2/players/player/{get_id}"

  headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': "cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b"
    }

  response = requests.request("GET", url, headers=headers)

  return response.json()

# json=id_player_info(21212)
# actual_name = json['api']['players'][1]['player_name']
# actual_name



In [ ]:
import requests
import pandas as pd
import json

def get_team_season_players(team_id, season):
    """ Extract json with all players info of a given team and season
        Args: team_id -- int, season (YYYY-YYYY)
    """
    url = f"https://api-football-v1.p.rapidapi.com/v2/players/squad/{team_id}/{season}"
    headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
    response = requests.request("GET", url, headers=headers)
    return response.json()

In [ ]:
def get_players_dict(team_id, season):
    """ Get dict of all players for given team and season where
         (key, value) -> (player_name, (player_id, p_firstname, p_lastname))
         Args: team_id -- int, season (YYYY-YYYY)
    """
    players_dict = {}
    json = get_team_season_players(team_id, season)['api']['players']
    for i in range(len(json)):
        players_dict[json[i]['player_name']] = (json[i]['player_id'], json[i]['firstname'], json[i]['lastname'])
    return players_dict

In [ ]:
def get_player_stats_df(player_id, team1_id, team2_id, year1, year2):
    """ Replace player_id arg with player_name and use the fns above to get the id
        Args: player_id (int), ...
    """
    url1 = f"https://api-football-v1.p.rapidapi.com/v3/players?id={player_id}&season={year1}"
    url2 = f"https://api-football-v1.p.rapidapi.com/v3/players?id={player_id}&season={year2}"
    headers = {
            'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
            'x-rapidapi-key': 'cb394d9372mshfc40fe51e5602b8p1a52d4jsn29044e42587b'}
    response1 = requests.request("GET", url1, headers=headers).json()
    response2 = requests.request("GET", url2, headers=headers).json()

    columns_to_remove = ['team.logo', 'league.id', 'league.logo', 'league.flag', 'games.number', 'games.position',
                     ]
    df1 = pd.json_normalize(response1['response'][0]['statistics'])
    df1 = df1[df1['team.id'] == team1_id]

    df2 = pd.json_normalize(response2['response'][0]['statistics'])
    df2 = df2[df2['team.id'] == team2_id]

    df = pd.concat([df1, df2], ignore_index=True)
    df = df.drop(columns=columns_to_remove)
    return df

In [ ]:
#here we make a function that will produce a data frame
#and will append to a data frame
players_list = []
player_id_list = []
team_to_list = []
team_to_id_list = []
team_from_list = []
team_from_id_list = []
year_before_list = []
year_after_list = []
isgood_transfer_list = []

#EMPTY DICT WE WILL FILL
#info_dict = {'player_name': players_list, 'player_id' : player_id_list,
#'team_from': team_from_list , 'team_to': team_to_list, 'team_from_id': team_from_id_list, 'team_to_id': team_to_id_list
 #            , 'year_before': year_before_list, 'year_after': year_after_list}

In [ ]:

def produceData(player_name, team_from, team_to, year_before, year_after, isgood_transfer):
    #from the team name, find the player, get their id and add it
    #getting the team ids
    team_1id = get_team_id(team_from)
    team_2id = get_team_id(team_to)
    season =  f'{year_before}-{year_before + 1}'
    print(season)
    #getting the player id
    player_id = get_player_id(player_name, team_from,season)
    #based on the player id get the name on file
    name_Df = id_player_info(player_id)
    actual_name = name_Df['api']['players'][1]['player_name']

    #Now lets append the data to a list and add it
    players_list.append(actual_name)
    player_id_list.append(player_id)
    team_to_list.append(team_to)
    team_from_list.append(team_from)
    year_before_list.append(year_before)
    year_after_list.append(year_after)
    team_from_id_list.append(team_1id)
    team_to_id_list.append(team_2id)
    isgood_transfer_list.append(isgood_transfer)


    #UPDATING THE DICT
    #return name



In [ ]:
info_dict = {'player_name': players_list, 'player_id' : player_id_list,
'team_from': team_from_list , 'team_to': team_to_list, 'team_from_id': team_from_id_list, 'team_to_id': team_to_id_list
             , 'year_before': year_before_list, 'year_after': year_after_list, 'isgood_transfer': isgood_transfer_list}

#GET SOME MORE FROM HERE
#https://www.90min.com/posts/best-transfers-of-2021-ranked

#EVERYTIME WE WANT TO ADD A PLAYER JUST CALL THE FUNCTION LIKE I DID BELOW
#ADD THE GOOD TRANSFERS HERE:
produceData('Ronaldo', 'Manchester United', 'Real Madrid', 2008,2010, True)
produceData('Bale', 'Tottenham', 'Real Madrid', 2012,2014, True)
produceData('K. Mbappé', 'Monaco', 'PSG', 2016, 2018, True)
produceData('S. Giménez', 'Cruz Azul', 'Feyenoord', 2021, 2023, True)
produceData('Haaland', 'Borussia Dortmund', 'Manchester City', 2021, 2023, True)
#MAKE SURE TO ADD THE DATA TO THE TEAMS
produceData('S. Haller', 'West Ham', 'Ajax', 2020, 2022, True)
produceData('H. Çalhanoğlu', 'AC Milan', 'Inter Milan', 2020, 2022, True)
produceData('R. Lewandowski', 'Lech Poznan', 'Borussia Dortmund', 2009, 2011, True)
produceData('Bruno Fernandes', 'Sporting CP', 'Manchester United', 2019, 2021, True)
produceData('L. Suárez', 'Barcelona', 'Atletico Madrid', 2019, 2021, True)
produceData('E. Džeko', 'Roma', 'Inter Milan', 2020, 2022, True)
produceData('T. Kroos', 'Bayern Munich', 'Real Madrid', 2013, 2015, True)
produceData('Mohamed Salah', 'Roma', 'Liverpool', 2016, 2018, True)
produceData('J. Stones', 'Everton', 'Manchester City', 2015, 2017, True)
produceData('S. Mané', 'South Hampton', 'Liverpool', 2015, 2017, True)


#ADDING SOME BAD TRANSFERS





2008-2009
FOUND
2012-2013
FOUND
2016-2017
FOUND
2021-2022
FOUND
2021-2022
FOUND
2020-2021
FOUND
2020-2021
FOUND
2009-2010
FOUND
2019-2020
FOUND
2019-2020
FOUND
2020-2021
FOUND
2013-2014
FOUND
2016-2017
FOUND
2015-2016
FOUND
2015-2016
FOUND


In [ ]:
#displaying the data
players_df = pd.DataFrame.from_dict(info_dict)
players_df

,player_name,player_id,team_from,team_to,team_from_id,team_to_id,year_before,year_after,isgood_transfer
0,Cristiano Ronaldo,874,Manchester United,Real Madrid,33,541,2008,2010,True
1,G. Bale,758,Tottenham,Real Madrid,47,541,2012,2014,True
2,K. Mbappé,278,Monaco,PSG,91,85,2016,2018,True
3,S. Giménez,94562,Cruz Azul,Feyenoord,2295,209,2021,2023,True
4,E. Haaland,1100,Borussia Dortmund,Manchester City,165,50,2021,2023,True
5,S. Haller,1826,West Ham,Ajax,48,425,2020,2022,True
6,H. Çalhanoğlu,1640,AC Milan,Inter Milan,489,505,2020,2022,True
7,R. Lewandowski,521,Lech Poznan,Borussia Dortmund,347,165,2009,2011,True
8,Bruno Fernandes,1485,Sporting CP,Manchester United,228,33,2019,2021,True
9,L. Suárez,157,Barcelona,Atletico Madrid,529,530,2019,2021,True


In [ ]:
import numpy as np

def get_stats_df(players_df):
  change_df = pd.DataFrame()

  for idx, row in players_df.iterrows():
    p_id = row['player_id']
    team_fromid = row['team_from_id']
    team_toid = row['team_to_id']
    year_before = row['year_before']
    year_after = row['year_after']

    df = get_player_stats_df(p_id, team_fromid, team_toid, year_before, year_after)
    numeric_df = df.select_dtypes(include=[np.number])
    summed_df = numeric_df.groupby('team.id').sum().reset_index()

    before_team_stats = summed_df[summed_df['team.id'] == team_fromid].drop('team.id', axis=1)
    after_team_stats = summed_df[summed_df['team.id'] == team_toid].drop('team.id', axis=1)

    # Subtract the "before team" stats from the "after team" stats
    change_df = pd.concat([change_df, after_team_stats.reset_index(drop=True) - before_team_stats.reset_index(drop=True)], ignore_index=True)
  return change_df

In [ ]:
change_df = get_stats_df(players_df)
change_df

,games.appearences,games.lineups,games.minutes,substitutes.in,substitutes.out,substitutes.bench,goals.total,cards.yellow,cards.yellowred,cards.red,...,dribbles.attempts,dribbles.success,fouls.drawn,fouls.committed,penalty.won,penalty.scored,penalty.missed,goals.conceded,league.season,penalty.commited
0,7.0,7.0,716.0,0.0,-3.0,-1.0,30.0,-1.0,-1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,0.0,13.0,3.0,3.0,6.0,-9.0,-8.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.0,7.0,885.0,-8.0,-13.0,-16.0,13.0,4.0,0.0,2.0,...,50.0,24.0,22.0,10.0,-1.0,1.0,0.0,NaN,NaN,NaN
3,-28.0,-3.0,-973.0,-25.0,-7.0,-26.0,9.0,-5.0,-1.0,0.0,...,-19.0,-9.0,4.0,-20.0,NaN,0.0,1.0,0.0,NaN,NaN
4,-14.0,-11.0,-1018.0,-3.0,-5.0,-2.0,-16.0,-3.0,0.0,0.0,...,-31.0,-13.0,-6.0,-15.0,NaN,-5.0,1.0,0.0,10.0,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12.0,9.0,473.0,3.0,11.0,4.0,-4.0,2.0,0.0,0.0,...,-40.0,-26.0,-13.0,31.0,NaN,0.0,0.0,0.0,8.0,NaN
7,14.0,14.0,1081.0,0.0,7.0,0.0,10.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20.0,17.0,1500.0,3.0,12.0,3.0,-5.0,0.0,-1.0,0.0,...,15.0,4.0,7.0,13.0,NaN,-7.0,2.0,NaN,NaN,NaN
9,7.0,-4.0,-408.0,11.0,7.0,16.0,-10.0,3.0,0.0,0.0,...,-29.0,-24.0,-11.0,2.0,NaN,1.0,1.0,NaN,NaN,NaN


In [ ]:
  # Test for griezmann from atletico madrid to barcelona 2018-2019
df = get_player_stats_df(56, 530, 529, 2018, 2019)
df

In [ ]:
import numpy as np
from IPython.display import display

numeric_columns = df.select_dtypes(include=[np.number])
# print(type(numeric_columns))
# display(numeric_columns)

# before_team_boolean = numeric_columns[numeric_columns['team.id'] == 530]
# after_team_boolean = numeric_columns[numeric_columns['team.id'] == 529]

summed_df = df.groupby('team.id').sum().reset_index()
summed_df

In [ ]:

before_team_stats = summed_df[summed_df['team.id'] == 530].drop('team.id', axis=1)
after_team_stats = summed_df[summed_df['team.id'] == 529].drop('team.id', axis=1)

# Subtract the "before team" stats from the "after team" stats
change_df = after_team_stats.reset_index(drop=True) - before_team_stats.reset_index(drop=True)
change_df